In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('project').getOrCreate()

In [2]:
from pyspark.sql.functions import length
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import NaiveBayes
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [3]:
df = spark.read.csv('../Arquivos/SMSSpamCollection', inferSchema=True, sep='\t')
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



In [4]:
df = df.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')

In [5]:
df.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [6]:
df = df.withColumn('length', length(df['text']))

In [7]:
df.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [8]:
df.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [9]:
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol='class', outputCol='label')

In [10]:
clean_up = VectorAssembler(inputCols=['tf_idf', 'length'], outputCol='features')

In [11]:
nb = NaiveBayes()

In [12]:
df_prep_pipe = Pipeline(stages=[ham_spam_to_numeric, tokenizer, stop_remove, count_vec, idf, clean_up])

In [13]:
cleaner = df_prep_pipe.fit(df)

In [14]:
clean_data = cleaner.transform(df)

In [15]:
clean_data = clean_data.select('label', 'features')

In [16]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [17]:
train, test = clean_data.randomSplit([0.7, 0.3])

In [18]:
spam_detector = nb.fit(train)

In [19]:
df.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [20]:
test_results = spam_detector.transform(test)

In [21]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,7,15,...|[-656.25906899705...|[1.0,2.7321750566...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-539.84179560431...|[1.0,1.9406019172...|       0.0|
|  0.0|(13424,[0,1,14,78...|[-707.68457777551...|[1.0,9.2108916101...|       0.0|
|  0.0|(13424,[0,1,15,20...|[-666.58016564781...|[1.0,1.5253977312...|       0.0|
|  0.0|(13424,[0,1,18,20...|[-851.30914479564...|[1.0,4.6226334148...|       0.0|
|  0.0|(13424,[0,1,23,63...|[-1306.4531171007...|[1.0,5.7192950649...|       0.0|
|  0.0|(13424,[0,1,31,43...|[-340.47626459612...|[1.0,5.7311387082...|       0.0|
|  0.0|(13424,[0,1,72,10...|[-682.05611263190...|[1.0,8.6062538030...|       0.0|
|  0.0|(13424,[0,1,416,6...|[-307.30859590898...|[0.99999999999995...|       0.0|
|  0.0|(13424,[0

In [22]:
acc_eval = MulticlassClassificationEvaluator()

In [23]:
acc = acc_eval.evaluate(test_results)

In [24]:
print(f"ACC of NB Model: {acc}")

ACC of NB Model: 0.9201206400472477
